第一个Jupyter！

分析排三开奖结果的另一种思路：
- 假设：开奖结果是按照顺序摇出的；
- 假设：开奖结果样本的按照奖期顺序从上到下排列；
- 将所有开奖结果变成一个一维向量，通过（4位）滑动窗口每次获取4个开奖球作为输入和输出（3个输入+1个输出）；
- 每个epoch都将滑动窗口以步长为1想后移动；

In [7]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import os
import collections
import time

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

# Target log path
logs_path = './logs'
writer = tf.summary.FileWriter(logs_path)

def read_data(path):
    """Load Dataset from File"""
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

将原始文件转成形如['2' '0' '2' '2' '4' '3' ... '' '' '']的格式，即将开奖球按照时间序列重新构造。
注意：cp.txt的最后不能有空行！否则做数字转换时会报错！

In [8]:
# Text file containing words for training
training_file = './data/cp.txt'

text = read_data(training_file)
# 将文件字符串转成一维向量，每个元素是三个连续的开奖号码，如123
list1 = np.array(text.split('\n'))
print(list1)
list2 = [int(item) for item in list1]
# 将一维的开奖号码转成二维，即将开奖号码的三个数字split成数组
arr1 = [[item[0:1], item[1:2], item[2:3]] for item in list1]
arr2 = np.array(arr1, dtype=np.int32)
print(arr2)
# 将上面的二维数组转成一维向量，表示历次的开奖球号的列表（不再体现明确显示奖期）
training_data = arr2.reshape((arr2.size))
print(training_data)
print("Loaded training data...")

['202' '243' '580' ... '718' '317' '192']
[[2 0 2]
 [2 4 3]
 [5 8 0]
 ...
 [7 1 8]
 [3 1 7]
 [1 9 2]]
[2 0 2 ... 1 9 2]
Loaded training data...


LSTM 模型：三个输入，一个输出

In [9]:
# 0-9
digital_size = 10

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, digital_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, digital_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([digital_size]))
}

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

ValueError: Variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-fd314b7620ef>", line 43, in RNN
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
  File "<ipython-input-3-fd314b7620ef>", line 49, in <module>
    pred = RNN(x, weights, biases)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


损失函数

In [10]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

ValueError: Variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/RMSProp/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-4-71413f148bc1>", line 3, in <module>
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


训练

In [6]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [ training_data[i]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([digital_size], dtype=float)
        symbols_out_onehot[training_data[offset+n_input]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc

        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = int(tf.argmax(onehot_pred, 1).eval())
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")

KeyboardInterrupt: 

预测

In [ ]:
    while True:
        prompt = "%s digitals: " % n_input
        three_digitals = input(prompt)
        three_digitals = three_digitals.strip()
        digitals = three_digitals.split(' ')
        if len(digitals) != n_input:
            continue
        try:
            symbols_in_keys = [digitals[i] for i in range(len(digitals))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                four_digitals = "%s %s" % (three_digitals, onehot_pred_index)
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(four_digitals)
        except:
            print("Digitals not in dictionary")

事实说明排三预测demo仅达到学习TensorFlow目的，预测就呵呵了 :-)
另外，py文件执行是好的，Jupyter中出现一些错误，有待学习完善。